In [1]:
# This code block is for automatic testing purposes, please ignore.
try:
    import openfermion
except:
    import os
    os.chdir('../src/')

# Demonstration of Givens rotation decomposition

In [2]:
import numpy
from openfermion.utils import givens_decomposition
from scipy.linalg import qr

numpy.set_printoptions(precision=3, linewidth=150, suppress=True)

A Slater determinant can be represented by an $m \times n$ matrix with orthonormal rows. Let's obtain such a matrix $Q$ using Scipy's QR decomposition algorithm.

In [3]:
m, n = (3, 6)

# Obtain a random matrix of orthonormal rows
x = numpy.random.randn(n, n)
y = numpy.random.randn(n, n)
A = x + 1j*y
Q, R = qr(A)
Q = Q[:m, :]
print(Q)

[[-0.311+0.062j -0.403-0.276j  0.097-0.412j  0.237-0.295j  0.091-0.504j  0.236-0.145j]
 [-0.289-0.156j -0.113+0.033j  0.262+0.525j -0.207+0.253j -0.212-0.122j  0.365-0.484j]
 [-0.330-0.72j  -0.018-0.099j -0.236-0.182j -0.185+0.005j -0.015+0.025j -0.460-0.164j]]


There exist unitary matrices $V$ and $U$ such that $VQU^\dagger$ is an $m \times n$ diagonal matrix. The matrix $VQ$ represents the same Slater determinant as $Q$ up to an overall phase (equal to $\det(V)$), but it has zeros in the upper diagonal, which saves some work in the preparation of the Slater determinant. The unitary $U$ can be written in the form

$$U = G_{N_G} \cdots G_2 G_1$$

where the $G_k$ are complex Givens rotations. We can obtain $V$, the $G_k$, and the diagonal entries of $VQU^\dagger$ using the `givens_decomposition` function.

In [4]:
# Get Givens decomposition of U
V, givens_rotations, diagonal = givens_decomposition(Q)
print(V)
print()
print(diagonal)

[[ 0.240-0.126j -0.439-0.47j  -0.689+0.195j]
 [ 0.900+0.065j -0.208+0.072j  0.357-0.101j]
 [ 0.335+0.j     0.683+0.268j -0.371+0.461j]]

[ 0.457+0.889j -0.916-0.4j    0.970-0.242j]


We didn't print the Givens rotations in the above block because it wouldn't look pretty. The Givens rotations are returned as a list of tuples of tuples. We will now iterate through the tuples of the list, and print the innermost tuple within each tuple as a string.

In [5]:
for parallel_set in givens_rotations:
    print(tuple(["{}, {}, {:.3f}, {:.3f}".format(i, j, theta, phi)
                 for i, j, theta, phi in parallel_set]))

('2, 3, 0.671, -2.644',)
('1, 2, 1.421, 0.281', '3, 4, 1.401, -1.617')
('0, 1, 0.683, -2.284', '2, 3, 0.704, 2.724', '4, 5, 1.204, 2.049')
('1, 2, 0.800, -2.721', '3, 4, 1.412, -2.494')
('2, 3, 1.112, -3.003',)


There are 5 tuples printed, and within each tuple, there are strings of the form 'i, j, theta, phi'. Each such string represents an innermost tuple, and it is a description of a complex Givens rotation of the coordinates $i$ and $j$ by angles $\theta$ and $\varphi$. The $2 \times 2$ matrix corresponding to this rotation is

$$
\begin{pmatrix}
\cos \theta & -e^{i \varphi} \sin \theta \\
\sin \theta & e^{i \varphi} \cos \theta
\end{pmatrix}.
$$

The fact that there are 5 tuples means that the circuit depth to prepare the Slater determinant corresponding to $Q$ (up to a phase) has depth 5. All of the rotations within the tuple can be performed in parallel; this is possible because the indices to be rotated are disjoint. For instance, in the third step, we can perform three rotations simultaneously, on coordinates $(0, 1)$, $(2, 3)$, and $(4, 5)$.

Let's check that $VQ$ has zeros in the upper right corner.

In [6]:
# Check that VQ has zeros in upper right corner
W = V.dot(Q)
print(W)

[[ 0.355+0.69j  -0.035+0.087j  0.301-0.385j  0.356-0.154j  0.000+0.j     0.000+0.j   ]
 [-0.403-0.176j -0.340-0.323j -0.081-0.496j  0.192-0.297j  0.165-0.427j  0.000-0.j   ]
 [ 0.194-0.048j -0.169-0.072j  0.242+0.25j  -0.064-0.069j -0.088-0.325j  0.704-0.432j]]


Now let's check the Givens decomposition. For each set of Givens rotations that can be performed in parallel, we construct the matrices corresponding to the Givens rotations and multiply them together. Then, we multiply $W = VQ$ repeatedly on the right by these matrices and check that the correct elements are zeroed out.

In [7]:
# Check the Givens decomposition
def expanded_givens(G, i, j, n):
    expanded_G = numpy.eye(n, dtype=complex)
    expanded_G[([i], [j]), (i, j)] = G
    return expanded_G

U = numpy.eye(n, dtype=complex)
for parallel_set in givens_rotations:
    print("Number of rotations to perform in parallel: {}".format(len(parallel_set)))
    combined_givens = numpy.eye(n, dtype=complex)
    for i, j, theta, phi in parallel_set:
        c = numpy.cos(theta)
        s = numpy.sin(theta)
        phase = numpy.exp(1.j * phi)
        G = numpy.array([[c, -phase * s],
                     [s, phase * c]], dtype=complex)
        expanded_G = expanded_givens(G, i, j, n)
        combined_givens = combined_givens.dot(expanded_G)
    W = W.dot(combined_givens.T.conj())
    U = combined_givens.dot(U)
    print(W)
    print()

Number of rotations to perform in parallel: 1
[[ 0.355+0.69j  -0.035+0.087j  0.384-0.491j  0.000-0.j     0.000+0.j     0.000+0.j   ]
 [-0.403-0.176j -0.340-0.323j -0.047-0.607j -0.072-0.032j  0.165-0.427j  0.000-0.j   ]
 [ 0.194-0.048j -0.169-0.072j  0.134+0.177j  0.220+0.179j -0.088-0.325j  0.704-0.432j]]

Number of rotations to perform in parallel: 2
[[ 0.355+0.69j  -0.236+0.585j  0.000-0.j     0.000-0.j     0.000-0.j     0.000+0.j   ]
 [-0.403-0.176j  0.160+0.516j -0.368-0.405j -0.425-0.187j  0.000+0.j     0.000-0.j   ]
 [ 0.194-0.048j -0.201-0.142j -0.140-0.051j -0.287+0.102j  0.272+0.164j  0.704-0.432j]]

Number of rotations to perform in parallel: 3
[[ 0.457+0.889j -0.000+0.j     0.000-0.j     0.000-0.j     0.000+0.j     0.000-0.j   ]
 [-0.000-0.j    -0.638-0.279j -0.483-0.531j -0.000-0.j     0.000-0.j     0.000+0.j   ]
 [ 0.000+0.j     0.308-0.077j -0.303-0.054j  0.141-0.015j  0.759+0.457j  0.000+0.j   ]]

Number of rotations to perform in parallel: 2
[[ 0.457+0.889j -0.000-0.j 

Finally, let's check that the final matrix, $VQU^\dagger$, is indeed diagonal, and that its entries match the ones returned by the function.

In [8]:
# Check the diagonal entries
D = numpy.zeros((m, n), dtype=complex)
D[numpy.diag_indices(m)] = diagonal
print("V * Q * U^\dagger matches the returned diagonal:")
print(numpy.all(numpy.isclose(D, V.dot(Q.dot(U.T.conj())))))

V * Q * U^\dagger matches the returned diagonal:
True
